In [ ]:
# only for google colab
!pip install bayesian-optimization

In [1]:
# only for google colab
from google.colab import drive
drive.mount('/content/drive')

In [1]:
import numpy as np
import pandas as pd
import random
import keras
import csv
import matplotlib.pyplot as plt
from keras.callbacks import EarlyStopping
from bayes_opt import BayesianOptimization
from keras.models import Model, load_model, model_from_json
from keras.layers import Input, Conv2D,MaxPooling2D,Dense, Flatten,Dropout
from keras.utils import np_utils

Using TensorFlow backend.


In [2]:
path = "X:/XXXX/XXXX" # root directory
groups = ["G2","notG2"] # class name
title="Results1" # name of bayesian optimization result file w/o extension

nb_ch = 1 # number of input image channels (1 or 2)
single_ch = 0 # if number of channel is 1, choose channel #0 or 1

nb_classes = len(groups)
if(path[-1]=="/"):
    path=path[:-1]

In [3]:
# load training and test data sets
x_train = np.load(path+"/data/x_train.npy")
y_train = np.load(path+"/data/y_train.npy")

x_test = np.load(path +"/data/X_test.npy")
y_test = np.load(path +"/data/Y_test.npy")

# normalize
x_train = x_train/255
x_test = x_test/255

y_train = np_utils.to_categorical(y_train,nb_classes)
y_test = np_utils.to_categorical(y_test,nb_classes)

# shuffle sequence of data
index=list(range(x_train.shape[0]))
index=random.sample(index,len(index))
x_train = x_train[index]
y_train = y_train[index]

# pick color channels
if nb_ch == 2:
    x_train=x_train[...,:2]
    x_test=x_test[...,:2]
else:
    x_train=x_train[...,single_ch]
    x_test=x_test[...,single_ch]
    x_train=x_train[...,None]
    x_test=x_test[...,None]

print(x_train.shape)
print(x_test.shape)

(1848, 135, 135, 1)
(113, 135, 135, 1)


In [5]:
# architecture of CNN
def buildModel(neurons,drop,nb_batch,hidden_layers):  
  
    nb_epoch=100

    neurons=int(neurons)
    hidden_layers=int(hidden_layers)

    # input layer
    inputs = Input(shape=x_train.shape[1:])

    x = Conv2D(neurons, (3, 3), padding='same', activation='relu')(inputs)
    x = MaxPooling2D((2, 2), padding='same')(x)

    # hidden layers
    if hidden_layers !=0:
        for i in range(1,hidden_layers+1):
            x = Conv2D(neurons*(2**i), (3, 3), padding='same', activation='relu')(x)
            x = MaxPooling2D((2, 2), padding='same')(x)

    x = Flatten()(x)
    x = Dense(neurons*(2**(hidden_layers+1)), activation='relu')(x)
    x = Dropout(drop)(x)

    # output
    predictions = Dense(nb_classes, activation='softmax')(x)

    # modeling
    model = Model(inputs=inputs, outputs=predictions)

    model.compile(optimizer='rmsprop',
                            loss='binary_crossentropy',
                            metrics=['accuracy'])
    
    early_stopping = EarlyStopping(patience=10, verbose=1)

    # learning
    hist = model.fit(x_train, y_train, validation_split=0.2, epochs=int(nb_epoch), batch_size=int(nb_batch),callbacks=[early_stopping])
    
    # evaluate the model with test data
    score = model.evaluate(x_test, y_test)
              
    print("loss : ",score[0])
    print("accuracy : ",score[1])
    
    acc.append(score[1])
    
    return score[0]*(-1)

In [5]:
# baeysian optimization
def bayesOpt():
    pbounds = {
        # define the range of each parameter
        'drop' : (0,0.9),
        "neurons" : (2,50),
        'nb_batch' : (4,100),
        'hidden_layers' : (0,6)
    }
    optimizer = BayesianOptimization(f=buildModel, pbounds=pbounds)
    
    #init_points: number of initial points
    #n_iter: number of iteration
    #acq:acquire function, ucb as default
    optimizer.maximize(init_points=10, n_iter=30, acq='ucb')
    return optimizer

In [ ]:
# run bayesian optimization
acc =[]
result = bayesOpt()
result.res
result.max

In [ ]:
# save results
all_params=[]
target=[]
for i in range(len(result.res)):
    all_params.append(result.res[i]["params"])
    target.append(result.res[i]["target"])

print("all_params:")
print(all_params)
print("target:")
print(target)

params_name=list(all_params[0].keys())

with open(path+'/'+title+'.csv', 'w') as f:
    label=["acc","target(loss)"]+params_name
    writer = csv.writer(f)
    writer.writerow(label)
    for i in range(len(all_params)):      
        eachparams=[acc[i],target[i]]+list(all_params[i].values())
        writer.writerow(eachparams)